In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import improve_utils as iu

2023-07-30 18:26:20.433835: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
cancer_gen_expr_model = tf.keras.models.load_model("Models//cancer_gen_expr_model")
cancer_gen_mut_model = tf.keras.models.load_model("Models//cancer_gen_mut_model")
cancer_dna_methy_model = tf.keras.models.load_model("Models//cancer_dna_methy_model")
cancer_gen_cnv_model = tf.keras.models.load_model("Models//cancer_gen_cnv_model")

2023-07-30 18:26:23.199516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 18:26:23.698089: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78910 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


In [4]:
import pickle

In [5]:
import numpy as np

In [6]:
with open("csa_data//drug_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [7]:
with open("csa_data//norm_adj_mat.pickle", "rb") as f:
    dict_adj_mat = pickle.load(f)

In [8]:
with open("csa_data//combo_gen_expr_cnv_norm.pickle", "rb") as f:
    combo_gen_expr_cnv_norm = pickle.load( f)

In [9]:
common_achs = list(combo_gen_expr_cnv_norm.keys())

In [10]:
from tensorflow.keras import backend as K

In [11]:
check = tf.keras.models.load_model("Models/combo_cdr_gcn_on_ctrpv2_0_drop_on_train_and_pred_sample_035")
check_full = tf.keras.models.load_model("Models/combo_cdr_gcn_on_ctrpv2_0_drop_on_train_and_pred")

In [12]:
def get_features(train_keep1):
    train_gcn_feats = []
    train_adj_list = []
    for drug_id in train_keep1["Drug_ID"].values:
        train_gcn_feats.append(dict_features[drug_id])
        train_adj_list.append(dict_adj_mat[drug_id])
    train_gcn_feats = np.array(train_gcn_feats).astype("float32")
    train_adj_list = np.array(train_adj_list).astype("float32")
    train_gcn_feats_omics = []
    for cnc_id in train_keep1["Cell_Line"].values:
        train_gcn_feats_omics.append(combo_gen_expr_cnv_norm[cnc_id])
    train_gcn_feats_omics = np.array(train_gcn_feats_omics).astype("float32")
    l2, l1 = train_keep1["Cell_Line"].values.reshape(-1,1), train_keep1["Cell_Line"].values.reshape(-1,1)
    y_train = train_keep1["AUC"].values.reshape(-1,1)
    return [[train_gcn_feats, train_adj_list, train_gcn_feats_omics, l2, l1], y_train]

In [13]:
gdsc_v2_all = iu.load_single_drug_response_data_v2(source = 'GDSCv2_all', split_file_name='GDSCv2_all.txt', y_col_name='AUC')[["improve_sample_id", "improve_chem_id", "auc"]]
gdsc_v2_all.columns = ["Cell_Line", "Drug_ID", "AUC"]
# gdsc_v1_all = gdsc_v1_all.sample(frac = 0.15)

Response data: (114644, 14)
Unique cells:  470
Unique drugs:  168


/lustre/work/statgrads/vpiyush2/DeepCDR_TF/improve_utils.py:187: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(improve_globals.y_file_path, sep=sep)


In [14]:
gdsc_v2_all.shape

(114644, 3)

In [15]:
list_df = [gdsc_v2_all[i:i+25000] for i in range(0,gdsc_v2_all.shape[0],25000)]

In [ ]:
# preds = []
# for i in range(0, 100): 
#     preds.append(get_one_preds())
#     K.clear_session()
#     gc.collect()

In [16]:
import gc

In [17]:
actual = []
preds = []
for batch in list_df: 
    features_gdsc_v2_all, target_gdsc_v2_all = get_features(batch)
    with tf.device('/gpu:0'):
        preds_gdsc_v2_all = check_full.predict(features_gdsc_v2_all, verbose = 1, batch_size = 64)
    actual.append(target_gdsc_v2_all)
    preds.append(preds_gdsc_v2_all)
    K.clear_session()
    gc.collect()

2023-07-30 18:26:38.544902: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-30 18:26:38.677147: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


229/229 [==============================] - 2s 7ms/step


In [18]:
actual = [inner for item in actual for inner in item]

In [19]:
preds = [inner for item in preds for inner in item]

In [ ]:
# _, tar = get_features(list_df[0])

In [ ]:
# list_df[0]

In [ ]:
# features_gdsc_v1_all, target_gdsc_v1_all = get_features(gdsc_v1_all)

In [ ]:
# preds_gdsc_v1_all = check_full.predict(features_gdsc_v1_all, verbose = 1, batch_size = 64)

In [20]:
from scipy.stats import pearsonr

In [22]:
r = pearsonr(preds_gdsc_v2_all[:,0], target_gdsc_v2_all[:,0])

In [23]:
R_sq = r[0]**2

In [24]:
r

(0.5281042644611252, 0.0)

In [25]:
R_sq

0.2788941141420261